In [1]:
# import pandas as pd
import MagmaPandas as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import meltInc.plotting as p
from functools import partial
from MagmaPandas.geochemistry.fO2 import fO2_QFM
from MagmaPandas.geochemistry.Kd_ol_melt import Kd_FeMg_vectorised
from MagmaPandas.geochemistry.Fe_redox import Fe_redox
from MagmaPandas.geochemistry.PEC import Fe_equilibrate

p.layout()
config = mp.configuration()
config.print()

Kd Fe-Mg ol-melt.......................toplis
Melt Fe3+/Fe2+........................borisov
Melt thermometer...............putirka2008_14
Volatile solubility model......IaconoMarziano


In [2]:
melt = mp.read_melt_inclusion('./melt.csv', index_col=['name'], total_col='total')
inclusions = melt.iloc[0:10,:]

In [3]:
forsterite_host = 0.9
P_bar = 1e3

In [4]:
w, F1, oc = melt.Fe_equilibrate(forsterite_host, P_bar)


/Users/thomas/Desktop/MagmaPandas/src/MagmaPandas/geochemistry/fO2.py:162: UserWarning: O'Neill fO2: temperatures above 1420K present
  w.warn("O'Neill fO2: temperatures above 1420K present")


T correction 05

In [ ]:
o_crystallised = []
for _, inclusion in melt.iterrows():
    *_, OC = Fe_equilibrate(inclusion, forsterite_host, P_bar)
    o_crystallised.append(OC)
o_crystallised

In [5]:
oc

name
PI054-07-06   -0.01180
PI054-07-05   -0.01600
PI056-04-02   -0.01915
PI116-04-01   -0.00655
PI056-05-01   -0.00890
PI089-03-01   -0.01545
PI074-05-01   -0.01020
PI052-02-01   -0.01375
PI056-04-03   -0.02355
PI085-04-02   -0.01190
PI052-03-04   -0.00605
PI085-04-05   -0.01655
PI085-01-01   -0.01075
PI085-01-06   -0.00965
PI085-03-02   -0.01330
PI052-03-02   -0.01095
PI085-03-01   -0.01380
PI085-01-05   -0.01260
PI085-02-01   -0.01845
PI052-05-01   -0.01455
dtype: float64

In [ ]:
o